# Import Keras and all the dependencies

In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from IPython.display import HTML

In [2]:
BATCH_SIZE = 32
IMAGE_SIZE = 256
CHANNELS=3
EPOCHS=50

In [3]:
path='/content/drive/MyDrive/Tomato classification'

# Importing data

In [4]:

d = tf.keras.preprocessing.image_dataset_from_directory(
    path,
    seed=123,
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

Found 14733 files belonging to 10 classes.


In [5]:
class_names = d.class_names
class_names

['Tomato_Bacterial_spot',
 'Tomato_Early_blight',
 'Tomato_Late_blight',
 'Tomato_Leaf_Mold',
 'Tomato_Septoria_leaf_spot',
 'Tomato_Spider_mites_Two_spotted_spider_mite',
 'Tomato__Target_Spot',
 'Tomato__Tomato_YellowLeaf__Curl_Virus',
 'Tomato__Tomato_mosaic_virus',
 'Tomato_healthy']

In [6]:
for image_batch, labels_batch in d.take(8):
    print(image_batch.shape)
    print(labels_batch.numpy())

(32, 256, 256, 3)
[3 9 7 5 5 1 7 4 7 1 4 3 0 0 4 3 8 3 9 0 4 2 9 7 7 1 9 7 6 5 0 9]
(32, 256, 256, 3)
[6 2 4 6 9 6 0 7 7 4 0 9 7 7 4 7 0 2 5 7 3 9 4 9 4 4 1 0 7 6 5 3]
(32, 256, 256, 3)
[4 7 6 7 9 4 3 9 4 3 5 9 3 8 1 1 0 7 0 2 5 7 6 0 0 0 2 3 7 5 7 0]
(32, 256, 256, 3)
[2 9 1 9 8 6 7 4 2 2 6 9 4 5 4 5 5 5 6 7 2 1 8 4 7 0 5 0 9 6 4 1]
(32, 256, 256, 3)
[6 3 7 0 7 7 1 4 9 9 1 5 9 6 5 4 2 9 0 5 4 0 4 6 7 0 0 8 9 0 9 7]
(32, 256, 256, 3)
[4 9 2 0 6 5 4 0 3 4 7 0 2 5 2 4 7 4 9 5 0 7 4 4 7 4 7 3 6 5 5 6]
(32, 256, 256, 3)
[7 8 8 4 9 5 0 2 5 4 3 5 4 9 7 7 1 0 4 6 7 9 9 4 6 7 4 1 4 3 1 2]
(32, 256, 256, 3)
[3 0 7 6 5 3 7 5 6 3 1 4 7 3 0 7 1 5 5 0 6 9 7 5 6 9 7 1 9 8 3 9]


In [7]:
plt.figure(figsize=(20, 20))
for image_batch, labels_batch in d.take(1):
    for i in range(12):
        ax = plt.subplot(3, 4, i + 1)
        plt.imshow(image_batch[i].numpy().astype("uint8"))
        plt.title(class_names[labels_batch[i]])
        plt.axis("off")

Output hidden; open in https://colab.research.google.com to view.

# Split the dataset


In [8]:
len(d)

461

In [9]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1

    ds_size = len(ds)

    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)

    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)

    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)

    return train_ds, val_ds, test_ds

In [10]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(d)


In [11]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

# Building the model

**Here we do the data augmentation using the methods such as RandomFlip, RandomRotation, Resizing, and Rescaling.**





In [12]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

In [13]:
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255),
])

In [14]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 10

model = models.Sequential([
    resize_and_rescale,
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

model.build(input_shape=input_shape)

In [15]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (32, 256, 256, 3)         0         
                                                                 
 conv2d (Conv2D)             (32, 254, 254, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (32, 127, 127, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (32, 125, 125, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (32, 62, 62, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (32, 60, 60, 64)         

In [16]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [18]:
history = model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1,
    epochs=50,
)

Epoch 1/50
368/368 [==============================] - 6s 17ms/step - loss: 3.2318e-05 - accuracy: 1.0000 - val_loss: 0.0753 - val_accuracy: 0.9891
Epoch 2/50
368/368 [==============================] - 6s 17ms/step - loss: 2.5496e-05 - accuracy: 1.0000 - val_loss: 0.0771 - val_accuracy: 0.9885
Epoch 3/50
368/368 [==============================] - 6s 17ms/step - loss: 2.0371e-05 - accuracy: 1.0000 - val_loss: 0.0792 - val_accuracy: 0.9891
Epoch 4/50
368/368 [==============================] - 6s 17ms/step - loss: 1.6249e-05 - accuracy: 1.0000 - val_loss: 0.0807 - val_accuracy: 0.9885
Epoch 5/50
368/368 [==============================] - 6s 17ms/step - loss: 1.3078e-05 - accuracy: 1.0000 - val_loss: 0.0814 - val_accuracy: 0.9885
Epoch 6/50
368/368 [==============================] - 6s 17ms/step - loss: 1.0700e-05 - accuracy: 1.0000 - val_loss: 0.0828 - val_accuracy: 0.9891
Epoch 7/50
368/368 [==============================] - 6s 17ms/step - loss: 8.5943e-06 - accuracy: 1.0000 - val_loss: 0

In [20]:
model.save("../tomatoes.h5")

In [21]:
scores = model.evaluate(test_ds)


47/47 [==============================] - 6s 11ms/step - loss: 0.0875 - accuracy: 0.9914


In [22]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

In [24]:
import numpy as np

plt.figure(figsize=(15, 15))
for images, labels in test_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))

        predicted_class, confidence = predict(model, images[i].numpy())
        actual_class = class_names[labels[i]]

        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")

        plt.axis("off")


Output hidden; open in https://colab.research.google.com to view.